In [ ]:
!pip install transformers datasets evaluate torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 53.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

In [ ]:
!pip install wandb

In [ ]:
# Initialisation automatique de wandb avec la clé API
import wandb

wandb.login(key="5bdedceb185db2df4939192bcd55853b49ed06db")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: junsts719 (junsts719-ecole-nationale-sup-rieure-polytechnique-de-ya) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# ========================
# 📚 Étape 2 : Préparation des données
# ========================
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/patient_feedback.csv')

In [ ]:
def map_sentiment(rating):
    if rating <= 2:
        return "negative"
    elif rating == 3:
        return "neutral"
    else:
        return "positive"

df["sentiment"] = df["rating"].apply(map_sentiment)

# Suppression des lignes avec des valeurs manquantes ou feedback vide
df = df.dropna(subset=["feedback_text", "sentiment"])
df = df[df["feedback_text"].str.strip() != ""]

# Mapping vers index numérique
label2id = {"negative": 0, "neutral": 1, "positive": 2}
id2label = {v: k for k, v in label2id.items()}
df["label"] = df["sentiment"].map(label2id)

# Conservation des colonnes pertinentes pour le dataset HuggingFace
df_final = df[["feedback_text", "label"]].reset_index(drop=True)
df_final.head(50)

,feedback_text,label
0,Slow lab.,1
1,Parking difficulty.,0
2,Long wait.,2
3,Excellent service.,1
4,Long wait.,2
5,Professional doctor.,1
6,Attentive staff.,0
7,Scheduling issues.,2
8,Scheduling issues.,2
9,Scheduling issues.,1


In [ ]:
!pip install scikit-learn

In [ ]:
# ========================
# 🤖 Étape 4 : Tokenisation + Fine-tuning SERENGETI
# ========================
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import DataCollatorWithPadding
import evaluate
from datasets import Dataset

# Create dataset from the dataframe
dataset = Dataset.from_pandas(df[["feedback_text", "label"]])

# Filter out empty strings before tokenization and train/test split
dataset = dataset.filter(lambda example: example["feedback_text"] != "", batched=False)

# Diviser en train/test avant la tokenisation
train_test_dataset = dataset.train_test_split(test_size=0.2)

tokenizer = AutoTokenizer.from_pretrained("UBC-NLP/serengeti")
model = AutoModelForSequenceClassification.from_pretrained(
    "UBC-NLP/serengeti",
    num_labels=3,
    id2label=id2label,
    label2id=label2id
)
def tokenize_function(examples):
    texts = examples["feedback_text"]
    if isinstance(texts, str):
        texts = [texts]
    return tokenizer(
        texts,
        padding="max_length",
        truncation=True,
        max_length=512,
        return_tensors=None
    )

# Tokeniser chaque split séparément, ne retire que 'feedback_text'
tokenized_train = train_test_dataset["train"].map(
    tokenize_function,
    batched=True,
    remove_columns=["feedback_text"]
)
tokenized_test = train_test_dataset["test"].map(
    tokenize_function,
    batched=True,
    remove_columns=["feedback_text"]
)


data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    return accuracy.compute(predictions=predictions, references=labels)

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_steps=500, # Save checkpoint every 500 steps
    save_total_limit=2 # Keep only the last 2 checkpoints
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Filter:   0%|          | 0/44987 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/514 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.80M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/18.0M [00:00<?, ?B/s]

...

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at UBC-NLP/serengeti and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/35989 [00:00<?, ? examples/s]

Map:   0%|          | 0/8998 [00:00<?, ? examples/s]

/tmp/ipython-input-8-325975196.py:73: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss


In [ ]:
# ========================
# 🔍 Étape 5 : Test avec nouveaux feedbacks (en Douala/Ewondo)
# ========================
from transformers import pipeline

classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)

# ✨ Feedbacks test (tu peux mettre du Ewondo ou Douala ici)
feedbacks = [
    "tata a meka ndolo na hospita",  # Ewondo
    "dokita a bongi",                # Douala
    "professional service and quick response",
    "temps d'attente très long",
    "mɛ́ rẽ́ yitsó na biɛ̂ na mbɔ̌ŋ"  # Bassa : « service professionnel et réponse rapide »
]

classifier(feedbacks)

It seems like the training process stopped due to a `RuntimeError` when trying to save a checkpoint. This often happens when there isn't enough disk space. I recommend checking your available disk space using the command below:

In [ ]:
!df -h